In [106]:
def recover_make_(dir_,file,trg):
    os.chdir(dir_)
    recover_ = pickle.load(open(file,'rb'))
    for obj in recover_:
        os.chdir(trg)
        if os.path.isdir(obj):
            dir_ = make_dir("{}/{}/".format(obj,'source'))[0]
            os.chdir(dir_)
            with open('Makefile','wt') as make_file:
                for line in recover_[obj]:
                    make_file.write(line)

In [100]:
from make import recover_make

In [113]:
def rename_file(name,to_repl):
    old_name = name
    if '\ ' in name:
        name = name.replace('\ ','_')
        old_name = old_name.replace('\ ',' ')
    elif '(' in name:
        name = name.replace('\(','_')
        name = name.replace('\)','')
        old_name = old_name.replace('\\','')
    if to_repl:
        if not os.path.isfile(name):
            os.rename(old_name,name)
    return name

In [9]:
import os
import pandas as pd
import numpy as np
import subprocess
import re
import pickle
from functools import reduce
from make import make_dir,recover_make,all_,comp_,clang_flags,make_proc,make_routine,mopt_routine,reg_name
from make import opt_bc,opt_routine,targ_routine,phony_routine,clean_routine,select_groups
opt_ = ["-O%s" %i for i in [0,1,2,3,'s','z']]

clang_ =["-O%s" %i for i in [0,1,2,3,'fast','s','z','g','4']]

frame = pd.DataFrame(columns=opt_,index=clang_)
absdir = "./../../../../Unix_Source/"
initdir = os.getcwd()+'/'
largedir = absdir+'unix_cmp'
smalldir = './../../../../Unix_Source/unix_small_2/'

all_divide = r'all\s*\:\s*'
w_out_space_divide = r'(?<!\\)\s+'
name_divide = r'.+\s*\:\s+.*\.[cpx]{1,3}.*'

def recover_make_(dir_,file,trg):
    os.chdir(dir_)
    recover_ = pickle.load(open(file,'rb'))
    recover_make(trg,recover_)

def rename_file(name,to_repl):
    old_name = name
    if '\ ' in name:
        name = name.replace('\ ','_')
        old_name = old_name.replace('\ ',' ')
    elif '(' in name:
        name = name.replace('\(','_')
        name = name.replace('\)','')
        old_name = old_name.replace('\\','')
    if to_repl:
        if not os.path.isfile(name):
            os.rename(old_name,name)
return name

def c_make_build(obj,flag='-Oz'):
    output = ["MAKEFILE      = Makefile\n",
              "RM            = rm -f\n",
              "CXX           = {}\n".format(obj['CXX']),
              "LD            = $(CXX)\n",
              "CXX_COMPFLAGS     = {}\n".format(clang_flags),
              "OTHER             = {}\n".format(obj['OTHER']),
              "OPT_FLAG          = {}\n".format(flag)
             ]
    #####TARGET LINES 
    output+=["\n"]*2
    for ind,target in enumerate(obj['TARGET']):
        output.append("TARGET_T{}      = {}\n".format(str(ind+1),target))
    output.append("\n")
    for ind,target in enumerate(obj['TARGET']):
        obj_str = reduce(lambda x,y: x+y+'.bc '+' \\\n\t',obj['TARGET'][target],'\\\n\t')[:-1]
        output.append("OBJS_T{} = {}\n".format((ind+1),obj_str))
    output.append('\n')
    ####SUFFIXES
    output+=[".SUFFIXES :\n",".SUFFIXES : .h .{} .ll .bc\n".format(' .'.join(obj['SUFFIX']))]
    for suff in list(obj['SUFFIX']):
            output+=[".{}.ll:\n".format(suff),
             '\t@echo "------------------"\n',
            "\t$(RM) $*.ll\n",
            "\t$(CXX) $(CXX_COMPFLAGS) $(OTHER) $< -o $*.ll\n"]
    output+=[".ll.bc:\n",
              "\t$(RM) $*.bc\n",
              "\topt $(OPT_FLAG) $< -o $*.bc"]
    ##main all
    output+=["\n"]*2
    output.append("all: "+' '.join(obj['TARGET']))
    output+=["\n"]*2
    for ind,target in  enumerate(obj['TARGET']):
        output.append("{}: $({})\n".format(target,"TARGET_T%i" %(ind+1)))
    output+=["\n"]*2
    for ind,target in  enumerate(obj['TARGET']):
        output.append("$(TARGET_T%i):  $(OBJS_T%i) $(MAKEFILE)\n" %(ind+1,ind+1))
        output.append('\t@echo "=================="\n')
        output.append("\t$(RM) $(TARGET_T%i)\n" %(ind+1))
        output.append("\t$(LD) -o $(TARGET_T%i) $(LDFLAGS) $(OBJS_T%i) $(LIBS) $(OTHER)\n" %(ind+1,ind+1))
        output.append('\t@echo  " "')
        output+=["\n"]*2
    output+=['cleano:\t        \n']
    for ind,target in  enumerate(obj['TARGET']):
        output.append('\t$(RM) $(OBJS_T%i)\n' %i(ind+1))
    output+=['\t$(RM) *.ii\n',
             '\t$(RM) *.s\n',
             '\t$(RM) *.ll\n',
             '\t$(RM) *.bc\n',
             '\t$(RM) core\n',
             '\n',
             'cleane:\n']
    for ind,target in  enumerate(obj['TARGET']):
        output.append('\t$(RM) $(TARGET_T%i)\n' %(ind+1))
    output+=['\n',
             'clean:\tcleano cleane\n']
    ###########MAIN DEPENDENCIES
    for target in obj['TARGET']:
        for sr in obj['TARGET'][target]:
            output.append('{}.ll     :$(MAKEFILE) {}\n'.format(sr,obj['TARGET'][target][sr]['C_LIKE']))
            output.append('{0}.bc     :$(MAKEFILE) {0}.ll\n'.format(sr))
        
    return output

def ch_flag(obj,flag):
    obj[6] = "OPT_FLAG          = {}\n".format(flag)
    return obj

def log_file(lines,name=initdir+'init_log.is'):
    with open(name,'a+') as file:
        for line in lines:
            if line[-1]!='\n':
                file.write(line+'\n')
            else:
                file.write(line)
def mk_make(dir_,ls,input_):
    make_files = {}
    os.chdir(dir_)
    for prg in ls:
        if os.path.isdir(prg):
            make_files[prg] = c_make_build(input_[prg])
    return make_files

In [90]:
def mk_make(dir_,ls,input_):
    make_files = {}
    os.chdir(dir_)
    for prg in ls:
        if os.path.isdir(prg):
            make_files[prg] = c_make_build(input_[prg])
    return make_files

In [4]:
recover_make_(initdir,'make_large.dir',largedir)
clang__ = re.compile(r'\t+clang\+{0,2}\s.*\-S -emit\-llvm')
__opt = re.compile(r'[\t+\s]+opt\s.*')
target__ = re.compile(r'\t+clang\+{0,2}\s(.+\.bc\s)+\-o')
src = {}
os.chdir(largedir)
for prg in os.listdir():

    src.setdefault(prg,{'CXX':'','OPTFLAG':'',
                 'OTHER':'','TARGET':{},'SUFFIX':set()})
    os.chdir(largedir)
    if os.path.isdir(prg):
        dir_ = make_dir("{}/{}/".format(prg,'source'))[0]
        os.chdir(dir_)
        loc_count = 0
        with open('Makefile','r') as make_file:
            cxx = {}
            bc = {}
            for line in make_file:
                if clang__.match(line):
                    to_split = re.split(clang_flags,line) 
                    to_split = re.split(w_out_space_divide,''.join(to_split))[1:-1]
                    src[prg]['CXX'] = to_split[0]
                    to_split = to_split[1:]
                    cxx[to_split[to_split.index('-o')+1]] = {'cc':None,}
                    cxx[to_split[to_split.index('-o')+1]]['cc'] = to_split[0]
                    src[prg]['OTHER'] = ' '.join(to_split[to_split.index('-o')+2:])
                elif __opt.match(line):
                    ll_split = re.split(w_out_space_divide,line)[1:-1]
                    bc[ll_split[-1]] = {'ll':None}
                    bc[ll_split[-1]]['ll'] = ll_split[ll_split.index('-o')-1]
                elif target__.match(line):
                    to_split = re.split(w_out_space_divide,line)[2:-1]
                    target = rename_file(to_split[to_split.index('-o')+1],False)
#                     target = to_split[to_split.index('-o')+1]
                    src[prg]['TARGET'][target] = {}
                    for sp in to_split[:to_split.index('-o')]:
                         cxx[bc[sp]['ll']]['cc'] = rename_file(cxx[bc[sp]['ll']]['cc'],True)
                         src[prg]['TARGET'][target][os.path.splitext(cxx[bc[sp]['ll']]['cc'])[0]] = {'C_LIKE':None,'R_LIKE':None}
                         src[prg]['TARGET'][target][os.path.splitext(cxx[bc[sp]['ll']]['cc'])[0]]['C_LIKE'] = cxx[bc[sp]['ll']]['cc']   
                         src[prg]['SUFFIX'].add(os.path.splitext(cxx[bc[sp]['ll']]['cc'])[1][1:])
                                
                    

FileNotFoundError: [Errno 2] No such file or directory: 'make_large.dir'

## c_LIke file build

In [91]:
make_files =mk_make(largedir,list(src),src)

In [92]:
err = []
res_opt = {'-O0':{},'-O1':{},'-O2':{},'-O3':{},'-Os':{},'-Oz':{}}
os.chdir(largedir)
for prg in make_files:
    os.chdir(largedir)
    dir_ = make_dir("{}/{}/".format(prg,'source'))[0]
    os.chdir(dir_)
    for option in opt_:
        with open('Makefile','wt') as mk_file:
            for line in ch_flag(make_files[prg],option):
                mk_file.write(line)
        make_proc(['make'],log_file,prg)
        res = -1
        try:
            for targ in src[prg]['TARGET']:
                res =subprocess.check_output("wc -c {}".format(targ), stderr=subprocess.STDOUT,
                                                       shell=True)
                res_opt[option][prg] = res
                print(res)
            make_proc(['make','clean'],log_file,prg)
        except Exception:
            err.append(prg)
            print(prg,targ)

b'24112 main\n'
b'24168 main\n'
b'24168 main\n'
b'24168 main\n'
b'24168 main\n'
b'24168 main\n'
b'8576 fileinfo\n'
b'12720 fileinfo\n'
b'12720 fileinfo\n'
b'12720 fileinfo\n'
b'12720 fileinfo\n'
b'8568 fileinfo\n'
b'12928 CfgReader\n'
b'12984 CfgReader\n'
b'12984 CfgReader\n'
b'12984 CfgReader\n'
b'12984 CfgReader\n'
b'12984 CfgReader\n'
b'18832 main\n'
b'18936 main\n'
b'18880 main\n'
b'18880 main\n'
b'18880 main\n'
b'14784 main\n'
b'57816 main\n'
b'57872 main\n'
b'57872 main\n'
b'57872 main\n'
b'57872 main\n'
b'53776 main\n'
b'8816 chrarray\n'
b'8816 chrarray\n'
b'8816 chrarray\n'
b'8816 chrarray\n'
b'8816 chrarray\n'
b'8816 chrarray\n'
b'89840 neural\n'
b'13064 tr\n'
b'85792 neural\n'
b'13064 tr\n'
b'85792 neural\n'
b'12960 tr\n'
b'85792 neural\n'
b'12960 tr\n'
b'85792 neural\n'
b'12960 tr\n'
b'85792 neural\n'
b'12960 tr\n'
b'28720 main\n'
b'28672 main\n'
b'28672 main\n'
b'28672 main\n'
b'28672 main\n'
b'28672 main\n'
b'14784 BinVertolker\n'
b'14784 BinVertolker\n'
b'14784 BinVertolk

b'13928 SMLMAIN\n'
b'13976 SMLMAIN\n'
b'13976 SMLMAIN\n'
b'13976 SMLMAIN\n'
b'13976 SMLMAIN\n'
b'13928 SMLMAIN\n'
b'19816 main\n'
b'19816 main\n'
b'19816 main\n'
b'19816 main\n'
b'19816 main\n'
b'19816 main\n'
b'19960 compress\n'
b'19632 decompress\n'
b'24160 compress\n'
b'23792 decompress\n'
b'28256 compress\n'
b'23792 decompress\n'
b'28200 compress\n'
b'27848 decompress\n'
b'28256 compress\n'
b'23792 decompress\n'
b'24104 compress\n'
b'23752 decompress\n'
b'14416 main\n'
b'14416 main\n'
b'14416 main\n'
b'14416 main\n'
b'14416 main\n'
b'14416 main\n'


In [17]:
dmp = {}
os.chdir(initdir+smalldir)
for file in os.listdir():
    os.chdir(initdir+smalldir)
    if os.path.isdir(file):
        dir_ = make_dir("{}/{}/".format(file,'source'))[0]
        os.chdir(dir_)
        with open('Makefile','r') as mkfile:
            dmp[file]  = mkfile.readlines()

In [20]:
# os.chdir(initdir)
# pickle.dump(dmp,open('small_files.dmp','wb'))

In [31]:
for file in os.listdir(initdir+smalldir):
    os.chdir(initdir+smalldir)
    if os.path.isdir(file):
        os.chdir(initdir+smalldir+"{}/{}/".format(file,"source"))
        count = 0 
        for source in os.listdir():
            if re.match("\.[cpCPX]{1,3}",os.path.splitext(source)[1]):
                count+=1
        if count!=1:
            print(file)
        



# All files got single source, all files got single clang string

In [ ]:
def c_make_build(obj,flag='-Oz'):
    output = ["MAKEFILE      = Makefile\n",
              "RM            = rm -f\n",
              "CXX           = {}\n".format(obj['CXX']),
              "LD            = $(CXX)\n",
              "CXX_COMPFLAGS     = {}\n".format(clang_flags),
              "OTHER             = {}\n".format(obj['OTHER']),
              "OPT_FLAG          = {}\n".format(flag)
             ]
    #####TARGET LINES 
    output+=["\n"]*2
    for ind,target in enumerate(obj['TARGET']):
        output.append("TARGET_T{}      = {}\n".format(str(ind+1),target))
    output.append("\n")
    for ind,target in enumerate(obj['TARGET']):
        obj_str = reduce(lambda x,y: x+y+'.bc '+' \\\n\t',obj['TARGET'],'\\\n\t')[:-1]
        output.append("OBJS_T{} = {}\n".format((ind+1),obj_str))
    output.append('\n')
    ####SUFFIXES
    output+=[".SUFFIXES :\n",".SUFFIXES : .h .{} .ll .bc\n".format(obj['SUFFIX'])]
    output+=[".{}.ll:\n".format(obj['SUFFIX']),
     '\t@echo "------------------"\n',
    "\t$(RM) $*.ll\n",
    "\t$(CXX) $(CXX_COMPFLAGS) $(OTHER) $< -o $*.ll\n"]
    output+=[".ll.bc:\n",
              "\t$(RM) $*.bc\n",
              "\topt $(OPT_FLAG) $< -o $*.bc"]
    ##main all
    output+=["\n"]*2
    output.append("all: "+' '.join(obj['TARGET']))
    output+=["\n"]*2
    for ind,target in  enumerate(obj['TARGET']):
        output.append("{}: $({})\n".format(target,"TARGET_T%i" %(ind+1)))
    output+=["\n"]*2
    for ind,target in  enumerate(obj['TARGET']):
        output.append("$(TARGET_T%i):  $(OBJS_T%i) $(MAKEFILE)\n" %(ind+1,ind+1))
        output.append('\t@echo "=================="\n')
        output.append("\t$(RM) $(TARGET_T%i)\n" %(ind+1))
        output.append("\t$(LD) -o $(TARGET_T%i) $(LDFLAGS) $(OBJS_T%i) $(LIBS) $(OTHER)\n" %(ind+1,ind+1))
        output.append('\t@echo  " "')
        output+=["\n"]*2
    output+=['cleano:\t        \n']
    for ind,target in  enumerate(obj['TARGET']):
        output.append('\t$(RM) $(OBJS_T%i)\n' %(ind+1))
    output+=['\t$(RM) *.ii\n',
             '\t$(RM) *.s\n',
             '\t$(RM) *.ll\n',
             '\t$(RM) *.bc\n',
             '\t$(RM) core\n',
             '\n',
             'cleane:\n']
    for ind,target in  enumerate(obj['TARGET']):
        output.append('\t$(RM) $(TARGET_T%i)\n' %(ind+1))
    output+=['\n',
             'clean:\tcleano cleane\n']
    ###########MAIN DEPENDENCIES
    for trg in obj['TARGET']:
        output.append('{0}.ll     :$(MAKEFILE) {0}.{1}\n'.format(trg,obj['SUFFIX']))
        output.append('{0}.bc     :$(MAKEFILE) {0}.ll\n'.format(trg))
        
    return output

In [105]:
for obj in recover_make_(initdir,'small_files.dmp',initdir+smalldir):
    print(obj)

ABC Payroll
Squareroot Algorithm
a find the nimbrer game
[ Doubly Link List Use of Data Structure ]
Students database
A Simple Web Page Generator
Value Arrays
Cube3D-Spot
LCD Writer
string token argument list
a Binary Search Example
Basic String Manipulation Tutorial
C++ Linked List program
calc gkt
Geometric Calculator
timeme.c
[[[ Basic Calculator ]]]
big companies simulator
Crypto
token strings and command lines


In [114]:
recover_make_(initdir,'small_files.dmp',initdir+smalldir)
clang__ = re.compile(r'\t+clang\+{0,2}\s.*\-S -emit\-llvm')
src = {}
for prg in os.listdir(initdir+smalldir):
    os.chdir(initdir+smalldir)
    if os.path.isdir(prg):
        src[prg] = {}
        dir_ = make_dir("{}/{}/".format(prg,'source'))[0]
        os.chdir(dir_)
        count = 0
        with open('Makefile','r') as make_file:
            for line in make_file:
                if clang__.match(line):
                    src[prg]['CXX'] = re.search("clang\+{0,2}",line)[0]
                    to_split = re.split(w_out_space_divide,line)
                    targ_ = rename_file(to_split[to_split.index('-o')-1],True)
                    src[prg]['TARGET'] = []
                    src[prg]['TARGET'].append(os.path.splitext(targ_)[0])
                    src[prg]['SUFFIX'] = os.path.splitext(targ_)[1][1:]
                    src[prg]['OTHER'] = line[re.search('[^\s]+\.[l]{2}',line).span()[1]+1:][:-2]
                                
                    

CPT234P3.cpp
sqrt.cpp
jeu1.cpp
doubly.cpp
start.cpp
webpagegenerator.cpp
valarray.cpp
Cube3D-Spot.cpp
lcd.cpp
solvers.c
BinarySearch.cpp
string_manip_tutorial.cpp
Linked_List.cpp
view.cpp
geometric.c
timeme.c
calculator.cpp
licencing.c
crypto.cpp
espace.c


In [115]:
make_files =mk_make(initdir+smalldir,list(src),src)
err = []
res_opt = {'-O0':{},'-O1':{},'-O2':{},'-O3':{},'-Os':{},'-Oz':{}}
for prg in make_files:
    os.chdir(initdir+smalldir)
    dir_ = make_dir("{}/{}/".format(prg,'source'))[0]
    os.chdir(dir_)
    for option in opt_:
        with open('Makefile','wt') as mk_file:
            for line in ch_flag(make_files[prg],option):
                mk_file.write(line)
        make_proc(['make'],log_file,prg)
        res = -1
        try:
            for targ in src[prg]['TARGET']:
                res =subprocess.check_output("wc -c {}".format(targ), stderr=subprocess.STDOUT,
                                                       shell=True)
                res_opt[option][prg] = res
                print(res)
            make_proc(['make','clean'],log_file,prg)
        except Exception:
            err.append(prg)
            print(prg,targ)

b'13448 CPT234P3\n'
b'13448 CPT234P3\n'
b'13448 CPT234P3\n'
b'13448 CPT234P3\n'
b'13448 CPT234P3\n'
b'13448 CPT234P3\n'
b'8960 sqrt\n'
b'8960 sqrt\n'
b'8960 sqrt\n'
b'8960 sqrt\n'
b'8960 sqrt\n'
b'8960 sqrt\n'
b'13152 jeu1\n'
b'13152 jeu1\n'
b'13152 jeu1\n'
b'13152 jeu1\n'
b'13152 jeu1\n'
b'13152 jeu1\n'
b'13240 doubly\n'
b'13296 doubly\n'
b'13296 doubly\n'
b'13296 doubly\n'
b'13296 doubly\n'
b'13296 doubly\n'
b'25672 start\n'
b'25792 start\n'
b'25792 start\n'
b'25792 start\n'
b'25792 start\n'
b'25792 start\n'
b'13936 webpagegenerator\n'
b'13992 webpagegenerator\n'
b'13992 webpagegenerator\n'
b'13992 webpagegenerator\n'
b'13992 webpagegenerator\n'
b'13992 webpagegenerator\n'
b'28280 valarray\n'
b'32040 valarray\n'
b'32040 valarray\n'
b'32040 valarray\n'
b'32040 valarray\n'
b'27944 valarray\n'
Cube3D-Spot Cube3D-Spot
Cube3D-Spot Cube3D-Spot
Cube3D-Spot Cube3D-Spot
Cube3D-Spot Cube3D-Spot
Cube3D-Spot Cube3D-Spot
Cube3D-Spot Cube3D-Spot
b'13072 lcd\n'
b'17256 lcd\n'
b'13160 lcd\n'
b'17256